In [16]:
from folktables import ACSDataSource
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import folktables
import numpy as np

from bestLS_hindsight import *
from OnlineRidgeRiver import *
from lean_adahedge import *
import matplotlib.pyplot as plt
from bestLS_hindsight_together import *
from oridge_alwaysactive_implementable import *


from folktables.load_acs import state_list

In [2]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, TargetEncoder, StandardScaler, OneHotEncoder
def numeric_scaler(df, cols):
    '''
    df: pandas dataframe
    numeric_cols: (array of strings) column names for numeric variables

    no return: does inplace operation
    '''
    df_new = df.copy()
    mmscaler = MinMaxScaler()
    df_new[cols] = mmscaler.fit_transform(df_new[cols])
    return df_new

def ordinal_encoder(df, cols): # similar to label encoder which only works for targets?
    '''
    Encode categorical into 0 ... n-1
    '''
    df_new = df.copy()
    ordinal_enc = OrdinalEncoder()
    df_new[cols] = ordinal_enc.fit_transform(df_new[cols])
    return df_new

def oh_sklearn(df, cols):
    pass
    # Issues with this operation as it doesnt preseve number of columns etc, the dummies method below works
    # df_new = df.copy()
    # oh_enc = OneHotEncoder()
    # df_new[cols] = oh_enc.fit_transform(df_new[cols])
    # return df_new

def one_hot(df, cols): # idk if sklearns one-hot encoder is similar
    """
    df: pandas DataFrame
    param: cols a list of columns to encode 
    return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

def target_encoder(df, x_cols, y_col):
    df_new = df.copy()
    enc_auto = TargetEncoder(smooth="auto", target_type="continuous")
    df_new[x_cols] = enc_auto.fit_transform(df_new[x_cols], df[y_col])
    return df_new

def target_encoder2(df, x_cols, y_col):
    df_new = df.copy()
    enc_auto = TargetEncoder(smooth="auto", target_type="continuous")
    df_new[x_cols] = enc_auto.fit_transform(df_new[x_cols], df[y_col])
    return df_new, enc_auto

In [10]:
def plot_ORidgevsAnh_together(cumreg_groupwise_oridge, Anh):
  Anhbetter_count = 0
  N = len(Anh.cuml_regret_curve)
  for gnum in range(N):  
    oridge_regret_g = cumreg_groupwise_oridge[gnum]
    Tg = len(oridge_regret_g) # number of rounds this group is active
    Anh_end = Anh.cuml_regret_curve[gnum][-1] # last time steps cumulative regret
    oridge_end = oridge_regret_g[-1]
    if Anh_end <= oridge_end:
      Anhst = "Yes"
      Anhbetter_count += 1
    else:
      Anhst = "No"
    print(f'''Group number {gnum}, Tg is {Tg} \n
    oridge end regret {oridge_end:.2f}, Anh end regret {Anh_end:.2f}, \n
    Anh better {Anhst} ''')
    time_steps = np.arange(1, Tg+1)
    plt.plot(time_steps, cumreg_groupwise_oridge[gnum],'-b', label="Online Ridge (OR)")
    plt.plot(time_steps, Anh.cuml_regret_curve[gnum], '-r', label="Anh with OR meta experts")
    plt.xlabel("Time slots")
    plt.ylabel("Regret")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
  print(f'Anh better for {Anhbetter_count} out of {N} groups') # caught a bug here!

In [18]:
df_te_scaled = pd.read_pickle("./target_encoded/dataframes/data_all_targetencoded_15cols")
df_te_scaled

,OCCP,WKHP,AGEP,SCHL,ST,JWTRNS,DRAT,COW,SEX,RELSHIPP,POBP,ENG,MAR,RAC1P,PINCP
0,0.174295,0.295918,0.025316,0.305521,0.525227,0.001086,0.000090,0.634268,0.003645,0.024891,0.440289,0.998451,0.001210,0.731267,0.039472
1,0.019586,0.397959,0.037975,0.303514,0.526608,0.440986,0.000244,0.566230,1.000000,0.023964,0.295825,0.998377,0.001598,0.189201,0.030967
2,0.198418,0.173469,0.025316,0.305698,0.531881,0.000343,0.000416,0.633992,0.003765,0.026136,0.364825,0.998644,0.001433,0.730600,0.030467
3,0.143561,0.051020,0.215190,0.302304,0.531881,0.410761,0.000416,0.633992,0.003765,0.026136,0.357630,0.998644,0.691216,0.730600,0.053479
4,0.169829,0.091837,0.025316,0.305521,0.525227,0.001086,0.000090,0.566516,0.995435,0.024891,0.512168,0.998451,0.001210,0.731267,0.009455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630162,0.584837,0.397959,0.291139,0.630562,0.002089,0.440826,0.000416,0.567354,0.003765,0.999952,0.097861,0.638492,0.001433,0.314840,0.399670
1630163,0.437459,0.397959,0.379747,0.806307,0.007309,0.440986,0.000861,0.637931,0.998820,1.000000,0.097829,0.254902,1.000000,0.314533,0.299615
1630164,0.232059,0.397959,0.367089,0.630562,0.002089,0.440826,0.000416,0.679348,0.003765,0.964693,0.097861,0.998644,0.998923,0.314840,0.092501
1630165,0.299907,0.295918,0.063291,0.630301,0.000000,0.857815,0.000244,0.360696,1.000000,0.276848,0.098585,0.641927,0.001598,0.315268,0.060483


In [7]:
import joblib
bls_undrop = joblib.load('./target_encoded/models/bestsqloss/bls_undropped_15cols.pkl')

In [11]:
bls_undrop.make_all_numpyarr()

In [ ]:
bls_undrop

In [9]:
len(bls_undrop.loss_experts_arr)
print(bls_undrop.best_sqloss)

[]


In [20]:
A_t = np.load('./target_encoded/nparrays/A_t12groups.npy')

N = A_t.shape[1] # 12 groups 11 + 1 always active
d = df_te_scaled.shape[1] # d uses all 15 columns
T = df_te_scaled.shape[0] # ~1.5 million points
print(N, d, T) 

X_dat_te_scaled = df_te_scaled.drop('PINCP', axis=1) #dropping the income column
y_dat_te_scaled = pd.DataFrame(df_te_scaled['PINCP']) # picking up only the income column for the target

print(X_dat_te_scaled.shape, y_dat_te_scaled.shape)


12 15 1569188
(1569188, 14) (1569188, 1)


In [27]:
sensitive_groups = ['SEX', 'RAC1P']
X_dat_te_drop =  X_dat_te_scaled.drop(sensitive_groups, axis=1)
y_dat_te_drop = y_dat_te_scaled

In [12]:
bls_undropped = joblib.load('./target_encoded/models/bestsqloss/bls_undropped_15cols.pkl')

In [13]:
bls_undropped.best_sqloss

[]

In [15]:
import pickle
with open('./target_encoded/models/Anh/Anh_undropped_15cols.pkl', 'rb') as f:
    Anh_undropped = pickle.load(f)


In [23]:
or_implementable_undropped = OnlineRidgeImplementable_alwaysactive(X_dat_te_scaled, y_dat_te_scaled) # undropped columns

100%|██████████| 1569188/1569188 [03:21<00:00, 7794.34it/s]


In [25]:
with open('./target_encoded/models/oridge_implementable/oridge_undropped15col.pkl', 'wb') as f:
    pickle.dump(or_implementable_undropped, f)

In [29]:
or_implementable_dropped = OnlineRidgeImplementable_alwaysactive(X_dat_te_drop, y_dat_te_drop) # undropped columns

100%|██████████| 1569188/1569188 [03:12<00:00, 8154.10it/s]


In [30]:
with open('./target_encoded/models/oridge_implementable/oridge_dropped15col.pkl', 'wb') as f:
    pickle.dump(or_implementable_dropped, f)